# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846830157049                   -0.70    4.0
  2   -7.852323922276       -2.26       -1.53    1.0
  3   -7.852646023982       -3.49       -2.52    3.0
  4   -7.852646676004       -6.19       -3.33    2.8
  5   -7.852646685831       -8.01       -4.74    1.0
  6   -7.852646686728       -9.05       -5.24    4.0
  7   -7.852646686729      -11.76       -5.87    1.0
  8   -7.852646686730      -12.41       -7.38    2.0
  9   -7.852646686730   +  -15.05       -7.65    2.8
 10   -7.852646686730      -15.05       -8.14    1.0
 11   -7.852646686730      -15.05       -9.54    2.0
 12   -7.852646686730   +    -Inf      -10.08    2.8
 13   -7.852646686730   +  -15.05      -10.92    1.2
 14   -7.852646686730   +    -Inf      -11.90    2.5
 15   -7.852646686730      -15.05      -12.22    1.2


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846853360730                   -0.70           4.2
  2   -7.852526428814       -2.25       -1.62   0.80    5.0
  3   -7.852609928822       -4.08       -2.79   0.80    5.2
  4   -7.852646573318       -4.44       -3.57   0.80    7.5
  5   -7.852646683593       -6.96       -4.55   0.80    9.0
  6   -7.852646686679       -8.51       -5.12   0.80   10.8
  7   -7.852646686725      -10.34       -6.18   0.80   11.8
  8   -7.852646686730      -11.35       -7.21   0.80   13.8
  9   -7.852646686730      -13.50       -7.86   0.80   15.5
 10   -7.852646686730   +  -14.27       -8.76   0.80   16.2
 11   -7.852646686730      -15.05      -10.02   0.80   17.8
 12   -7.852646686730   +  -15.05      -10.82   0.80   20.5
 13   -7.852646686730      -15.05      -11.24   0.80   22.2
 14   -7.852646686730      -14.75      -12.28   0.80   23.0


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.392439e+01     3.308221e+00
 * time: 0.486102819442749
     1     1.658430e+00     2.177491e+00
 * time: 0.7047569751739502
     2    -1.736543e+00     2.082497e+00
 * time: 0.7290928363800049
     3    -3.643498e+00     2.032651e+00
 * time: 0.7650718688964844
     4    -5.316299e+00     1.552365e+00
 * time: 0.7995760440826416
     5    -6.801007e+00     9.814098e-01
 * time: 0.8370518684387207
     6    -6.959589e+00     1.522761e+00
 * time: 0.8627099990844727
     7    -7.406620e+00     1.071996e+00
 * time: 0.8875949382781982
     8    -7.614683e+00     2.977380e-01
 * time: 0.9240870475769043
     9    -7.703597e+00     1.959925e-01
 * time: 0.947861909866333
    10    -7.760399e+00     1.019701e-01
 * time: 0.9746208190917969
    11    -7.812143e+00     1.963673e-01
 * time: 1.0006158351898193
    12    -7.833373e+00     4.727438e-02
 * time: 1.0276439189910889
    13    -7.846697e+00     4.517344e-02
 * time: 1.054075956344

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846838018141                   -0.70    4.2
  2   -7.852321380332       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686706                   -2.54
  2   -7.852646686730      -10.62       -5.88
  3   -7.852646686730   +  -14.75      -12.56


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.1737076832317915e-13
|ρ_newton - ρ_scfv| = 2.172550854523969e-13
|ρ_newton - ρ_dm|   = 7.124118521879678e-10
